# Programming Assignment 5: Value Iteration


## Collaboration statement

List all collaborators by name, including other students and/or course staff (e.g. TAs, UCAs). If you collaborated with no one on this assignment, write the following: ``"I did not collaborate with anyone on this assignment."``

**Collaborators: Rajiv Swamy (UCA), Yongwei Che (UCA), Alfred Ripoll**

## Problem Setting

**Gridworld**: In this assignment, we use the popular RL toolkit  [OpenAI Gym](https://gym.openai.com/)  to implement _value iteration_ on a gridworld environment (Lecture 21). In the gridworld environment we consider, an agent needs to navigate on a 2-D plane by using 4 actions (left, right, up, and down). It starts at a designated state (starting point "S"), and needs to try and reach a goal state ("G"). The agent controls the movement of a character in a grid world. Some tiles on the grid are walkable ("S", "G", and "F"), and others ("H") lead to the agent falling into the water, which ends the episode. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. The agent stays in the same position if the transition would otherwise have put them out of bounds.

**Rewards**: The environment is such that reaching the goal state gives the agent a `+1` reward, and it gets a `0` reward in other states (detailed description below). Hence, reaching the goal state is equivalent to maximizing the rewards it can accumulate.

**Uncertainty**: The environment also has some uncertainty. That is, the movement direction of the agent is uncertain and only partially depends on the chosen direction. In our setting (detailed below), each tile in the grid is "slippery", which corresponds to the following uncertainty; if the agent chose to move up/down, it will be able to do so with probability `1/3`, but with probability `2/3` it will move right/left (`1/3` right, `1/3` left). Similarly for the case that the agent chose to move right/left, it might move up/down instead. However, you will always move by 1 tile only (i.e., no "jumps").

**Example**


In a) the agent (penguin) tries to walk down, in b) it tries to walk right, and c) shows a potential pitfall where just by walking by a hole "H" (and not directly into it), the agent still has a `1/3` chance to fall into the hole, which ends the episode with no reward.


Note that:
* The specific transition probabilities don't matter much here, but it is important to keep in mind is that the action you choose to move towards only partially determined where you actually move, due to the uncertianty ("slipperiness") of the environment.

* Recall that while you can see the whole gridworld (all the states), the agent only has information about its current state!

## Environment: Frozen Lake
Winter is here. You and your friends are tossing around a frisbee at the park when you make a wild throw that leaves the frisbee stranded out in the middle of
a lake. The water is mostly frozen ("F"), but there are a few holes ("H") where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend, and sometimes slip into another, adjacent tile! The surface is described using a grid like the following $8 \times 8$ example:

    SFFFFFFF
    FFFFFFFF
    FFFHFFFF
    FFFFFHFF
    FFFHFFFF
    FHHFFFHF
    FHFFHFHF
    FFFHFFFG


    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located


The episode ends when you reach the goal or fall in a hole "H". You receive a reward of 1 if you reach the goal, and 0 otherwise. The steps that you can make are one of the following: LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3.

## Visualization
It is easier to understand how this environment behaves if you see it first.  

**HINT:** We **strongly recommend** that before implementing anything, you start by first running all cells of this notebook, and scroll down to the cell under **ACT7** see how a **totally-random** agent moves around the frozen lake.

You will see it fail many times. In what follows, we will implement a much better agent that can navigate to the goal. **You can use the visualization to see how your agent performs, where it fails, and debug issues that arise during development.**
      

## Notation

Mapping of the OpenAI gym constructs to the notation used in lectures:
* The value function $V(s)$, is implemented here as a small array, of size corrsponsing to number of states, such that `V[s]` contains the value $V(s)$.
* The transition probabilities of the environment $P(s'|s,a)$ correspond to the gym object `env.P` described below.
* In lectures, the reward function $r(a|s, s')$ signified  taking action $a$ while in state $s$ moving to state $s'$. However, in our setting here the reward fixed into the environment is such that $r(a|s=\text{'G'}, s')=1$ and $r(a|s \neq\text{'G'}, s')=0$ for any $a, s'$. In other words, in this setting we only get the reward if we *actually reach* the goal state.

### First, install dependencies (takes ~1 min).

In [67]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay pygame > /dev/null 2>&1
!apt-get install -y xvfb libav-tools python-opengl ffmpeg > /dev/null 2>&1

In [68]:
import gym
import numpy as np
import random
from IPython.display import clear_output
import time
import pygame
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [69]:
# setting up dummy display driver for Colab
import os
os.environ['SDL_VIDEODRIVER']='dummy'
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [70]:
# first, we set some useful parameters:
gamma = 0.99        # discount factor
theta = 0.000001    # precision of value_iteration

# let's set up the frozen lake environment.
env = gym.make("FrozenLake8x8-v1", new_step_api=True) # create the environment
env = env.unwrapped # unwrap it to have additional information from it

# spaces dimension
num_actions = env.action_space.n # we can move up/down/left/right, i.e., 4 actions.
num_states = env.observation_space.n # we have one state per tile in the grid (64 states)

# NOTE, the transition probabilities we mentioned above (i.e., the "slipperiness"), are already given
# in the env.P object. Specifically, env.P are the transition probabilities (dictionary dict of dicts of lists)
#           P[s][a] == [(probability, s', reward, done),
#                       (probability, s', reward, done),
#                       (probability, s', reward, done), ]

# As an example, for state s=0 (the starting tile "S"),and action a=0 (LEFT), you will have:
#           P[0][0] == [(1/3, 0, 0, False), (1/3, 0, 0, False), (1/3, 4, 0, False)]
# since:
# with probability 1/3 you'll move UP (but there's no up, so you'll stay put at s=0),
# with probability 1/3 you'll move LEFT (but there's no left, so you'll stay put at s=0),
# with probability 1/3 you'll move DOWN (so you'll get to the tile below, which is s'=8).


## Value-Iteration algorithm

We will now implement the Value Iteration algorithm you have seen in class. The pseudo-code is given below.

### Pseudo-code
1. Parameter: a small threshold $\theta>0$ determining accuracy of estimation.
1. Initialize $V(s)=-\frac{M}{1-\gamma}$, for all $s\in\mathbb{S}$, where $M$ is the upper bound on the absolute value of immediate rewards.
1. **Loop**:
    1. $\Delta \leftarrow 0$.
    1. **Loop for each** $s\in\mathbb{S}$:
        1. $v\leftarrow V(s)$.
        1. $V(s) \leftarrow \max_a\sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr]$.
        1. $\Delta\leftarrow \max(\Delta, \lvert v - V(s) \rvert)$.
1. **until** $\Delta < \theta$.
1. Output a deterministic policy, $\pi\approx\pi_*$, such that
$$ \pi(s) = \text{argmax}_a \sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr].$$

## ACTS 1-2: Computing the sum & argmax of Value-Iteration.

Compute the inner sum $$\sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr],$$ given $V, s, a, \gamma$.

In [71]:
# ACT 1: compute the inner sum of the equations above, given:
        # Value function V (an array of size num_states),
        # State s, Action a, and discount factor gamma.
        # Arguments (V, s, a, gamma) should not be mutated in this ACT

def sum_over_states(V, s, a, gamma):
    sum_val = 0
    # recall that env.P is a list of tuples for (s,a): [(prob, s', r, done),...] (See block above for more details!)
    # where 'prob' is the transition probability P(s'|s,a).
    # also, since r is associated with the states, you can think of the sum as only traversing states s'.
    # therefore, all is needed is to iterate over each possible transition to state s'
    # from the pair (s,a), and compute the sum value (as in the pseudo-code).
    ### YOUR CODE GOES HERE
    num = 0
    for step in env.P[s][a]:
      prob = env.P[s][a][num][0]
      reward = env.P[s][a][num][2]
      vals = V[env.P[s][a][num][1]]
      sum = prob * ((reward) + (gamma*vals))
      sum_val += sum
      num += 1
    return sum_val

We now compute the argmax operation, in which we will update $\pi(a|s)$ where $s$ is the given state, and $a$ is the maximizing action! To obtain it, you should use the function above `sum_over_states`.

**HINT**:
In the scaffold code for ACT5-6, pi is initialized to a zero matrix. If all the ACTs are implemented correctly, then the optimal policy pi returned by the `value_iteration` function (from ACT5-6) should have the following property: each row in pi should contain all zeros except at one index, where it contains a 1 (i.e. the index corresponding to the maximizing action for the state corresponding to that row).

Suppose we had a simple environment with 2 states and 3 actions. Then, our pi would be a 2 by 3 matrix. Suppose the following pi matrix was returned by `value_iteration`:

$$
\pi =
\begin{bmatrix}
  1 & 0 & 0 \\
  0 & 0 & 1 \\
\end{bmatrix}
$$

This would mean that when we're in state 0 (i.e. row index 0), we should take action 0 (i.e. column index 0 contains a 1) and when we're in state 1 (i.e. row index 1), we should take action 2 (i.e. column index 2 contains a 1).

Now going back to `argmax_over_actions` in ACT2, Given state s, suppose action a maximizes the inner sum. How should pi be updated? (You can assume pi[s] contains all zeros, from the scaffold code in ACT5-6).

In [82]:
# ACT 2: update the action which can take us to a higher valued state, given:
        # Value function V (an array of size num_states),
        # Policy array pi (a matrix of shape [num_states, num_actions], you can assume it contains all zero entries),
        # State s, and discount factor gamma.
        # Return the updated pi matrix and max_val (i.e. the value of taking the maximizing action from state s)

def argmax_over_actions(V, pi, s, gamma):
    max_val = float('-inf')
    max_action = 0
    for action in range(num_actions):
      new_val = sum_over_states(V, s, action, gamma)
      if new_val > max_val:
        max_action = action
        max_val = new_val
    pi[s, max_action] = 1
    return pi, max_val

## ACT 3: Computing the Bellman update

We now perform a single iteration of update to the value function $V$, given state $s$, and discount factor $\gamma$.
You may call a function you have implemented above, and output the difference between the new, updated value for this state, and the previous value (denoted as `delta` in the pseudo-code above).

In [83]:
# ACT 3: update the Value function for state s, that is: V[s], by taking action which maximizes current value. Given:
        # Value function V (an array of size num_states),
        # State s, and discount factor gamma.
# V should be updated in this ACT

def bellman_optimality_update(V, s, gamma):
    pi = np.zeros((num_states, num_actions))
    delta = 0
    prev_val = V[s]
    ### YOUR CODE GOES HERE
    # ACT3a: call a function implemented above, to get the action which maximizes current value.
    bell_pi, new_maxval = argmax_over_actions(V, pi, s, gamma)
        # ACT3b: update value V[s]
    V[s] = new_maxval
        # ACT 3c: compute and output delta.
    delta = abs(prev_val - V[s])
    return delta

## ACT 4: Initialize V
Initialize $V$ such that for all states, $V[s] = -\frac{M}{1-\gamma}$, where $M$ is the upper bound on the absolute value of immediate rewards to be filled in.

In [84]:
# ACT 4: Initialize V
      # length_V gives the size of the vector V
def init_V(gamma, M, length_V):
    ### YOUR CODE GOES HERE
    V = np.zeros(length_V)
    for state in range(num_states):
      V[state] = -1 * (M / (1-gamma))
    return V

## ACT 5-6: Value-iteration (putting it all together)

In [85]:
# Now, let's put it all together. Recall that we are given:
        # discount factor gamma, and wanted precision theta.
def value_iteration(gamma, theta):
    # Initialize V with init_V function
    ### YOUR CODE GOES HERE
    M = 1
    V = init_V(gamma, M, num_states)
    # ACT 5: construct the main loop,
    #        iteratively calling the bellman update,
    #        and break when sufficient accuracy was reached.
    while True:
        delta = 0
        ### YOUR CODE GOES HERE
        for state in range(num_states):
          delta1 = bellman_optimality_update(V, state, gamma)
          maxdelta = max(delta, delta1)
        if maxdelta < theta:
          break
    pi = np.zeros((num_states, num_actions))
    # ACT 6: Extract optimal policy using the
    # argmax_over_actions function defined in ACT 2
    ### YOUR CODE GOES HERE
    for state in range(num_states):
      pi, V2 = argmax_over_actions(V, pi, state, gamma)
    # output optimal value funtion, optimal policy
    return V, pi


## Helper functions (**no action needed**)
The next two functions `show_state` and `pretty_print` are helper functions we provide that print and visualize state information.

In [86]:
def show_state(env, trial=0, step=0):
    plt.figure(5)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title(f"Trial: {trial} | Step: {step}")
    plt.axis('off')
    clear_output(wait=True)
    display.display(plt.gcf())

In [87]:
def pretty_print(i_episode, t, done, reward, total_rewards, debug=False):
  if debug:
    show_state(env, i_episode, t)
    if done:
        print('')
        if reward == 1:
            total_rewards += 1
            print(">> Success!! got the goal")
        else:
            print(">> Failed!! fell into a hole")
        time.sleep(3), clear_output(wait=True)
    else:
      time.sleep(.5)
    it = i_episode + 1
    print(f"Avg reward so far = {(total_rewards / it):.2f}")
  else:
    if done:
      if reward == 1:
        total_rewards += 1
      it = i_episode + 1
      if it%10 == 0:
        print(f"Avg reward obtained in iteration {it} = {(total_rewards / it):.2f}")
  return total_rewards

## ACT 7: Call agent, and evaluate results

Below is the main loop of our setting: first, we call the `value_iteration` function, and obtain the optimal policy.
Then, we run in many episodes (or trials) of the algorithm. In each trial, we start at the starting state "S". We then ask the agent what action to go towards. We make a step in the frozen lake with the chosen action (but may not move there because the lake is slippery!), our next state is given after we call `env.step(action)`.

*You can expect to get an average reward in the range (0.8,1) upon correct implementation.*

**Note**: While running over 100 episodes, set `debug = False` to speed up your execution

In [89]:
# note that you can first try it out with a random agent, by setting this variable to True.
random_agent = False
episodes = 100

# When `debug` is set to True, you can see visualizations of the agent's actions
debug = False

if not random_agent:
    # We have called the value_iteration function:
    V_, pi = value_iteration(gamma, theta)
    # note that we don't need the value function V anymore, as we already extracted the optimal policy!

total_rewards = 0
for i_episode in range(episodes):
      state = env.reset()
      t = 0
      while True:
          t+=1
          # your agent picks an action:
          if random_agent:
            action = env.action_space.sample() # totally random action !
          else:
            # ACT 7: get the best action according to optimal policy `pi`, and current state `state`.
            action = 0
            ### YOUR CODE GOES HERE
            action = np.argwhere(pi[state])[0][0]
          # make a step according to policy
          state, reward, terminated, truncated, info = env.step(action)
          # the above variables are:
                # state (object): agent's observation, current state (new state we've reached after the step we took)
                # reward (float) : amount of reward returned after previous action
                # done (bool): whether the episode has ended
                # terminated (bool): whether a `terminal state` is reached (only True if the state is the Goal or a Hole!)
                # truncated (bool): whether a truncation condition outside the scope of the MDP is satisfied (eg. timelimit/agent physically going out of bounds)
                # info (dict): contains auxiliary diagnostic information (might be helpful for debugging, though not used in this assignment)

          done = truncated or terminated # we are done in either case

          # visualization
          total_rewards = pretty_print(i_episode, t, done, reward, total_rewards, debug=debug)
          if done:
            break

env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Avg reward obtained in iteration 10 = 0.80
Avg reward obtained in iteration 20 = 0.90
Avg reward obtained in iteration 30 = 0.87
Avg reward obtained in iteration 40 = 0.82
Avg reward obtained in iteration 50 = 0.80
Avg reward obtained in iteration 60 = 0.80
Avg reward obtained in iteration 70 = 0.81
Avg reward obtained in iteration 80 = 0.82
Avg reward obtained in iteration 90 = 0.82
Avg reward obtained in iteration 100 = 0.81


## Conceptual Questions (ACTs 8-10)

Notice that in your implementation above, an agent that follows the value-iteration policy performs much better than an agent that follows the random policy (you can run both variations by setting the `random_agent` variable to `True/False`).

Please **briefly** answer the following questions (1-2 sentences per question suffices).

#### **ACT8:** Does the value-iteration policy *always* obtains reward=1? Explain why/why not.
#### <span style='color:cyan'>  **Answer:** </span> The value iteration policy does not always obtain the reward because the reward is sparse in relation to the amount of states, so in some iterations of this case, the agent may fall into a hole.

#### **ACT9:** Will the random policy *always* obtain reward=0? Explain why/why not.
#### <span style='color:cyan'>  **Answer:** </span> The random policy will not always obtain reward 0. As small as it is, there is a nonzero chance that the agent makes all of the right random moves and obtains the goal by pure chance.

#### **ACT10:** When averaged over many episodes, will the value-iteration policy always obtain a larger reward than the random policy? Explain why/why not (no need to prove it, just state the appropriate claim shown in lecture and breifly explain how it implies this statement).
#### <span style='color:cyan'>  **Answer:** </span> Yes, it will always obtain a reward greater than that of a random policy because it is designed to converge at the 'optimal' policy, or the policy that maximizes the reward it can get.